In [ ]:
import numpy as np

In [ ]:
#FUNCION IM2COL EN PYTHON (FUNCIONES PARA PODER HACER LA CONVOLUCION MUCHO MÁS RAPIDO)
#FUNCION PARA UNA SOLA IMAGEN DE 2 DIMENSIONES
def im2Dcol(A, blocksize, stepsize=1):
    # Parameters
    M,N = A.shape
    col_extent = N - blocksize[1] + 1
    row_extent = M - blocksize[0] + 1
    
    # Get Starting block indices
    start_idx = (np.arange(blocksize[0])[:,None]*N + np.arange(blocksize[1])).T
    
    # Get offsetted indices across the height and width of input array
    offset_idx = (np.arange(row_extent)[:,None]*N + np.arange(col_extent)).T
    
    # Get all actual indices & index into input array for final output
    return np.take(A,start_idx.ravel()[:,None] + offset_idx.ravel()[::stepsize])

#FUNCION DE EXPANSION PARA UN CONJUNTO DE IMAGENES DE C CANALES
def imNDcol(A, blocksize, stepsize=1):
  m, n_C, n_H, n_W = A.shape
  #blocksize = [3,2,2]
  col_extent = n_W - blocksize[2] + 1
  row_extent = n_H - blocksize[1] + 1
      
  # Get Starting block indices
  start_idx = (np.arange(blocksize[1])[:,None]*n_W + np.arange(blocksize[2])).T
  start_idx = np.broadcast_to(start_idx, (1,n_C,blocksize[1], blocksize[2]))
  idx_s = np.arange(0,n_W*n_W*n_C,n_W*n_W)[:, None]
  idx_s = idx_s[None, :]
  idx_s = idx_s[:,:,np.newaxis]
  start_idx = start_idx + idx_s 

  # Get offsetted indices across the height and width of input array
  offset_idx = (np.arange(row_extent)[:,None]*n_W + np.arange(col_extent)).T
  h_o, w_o = offset_idx.shape
  offset_idx = np.broadcast_to(offset_idx, (m, 1, h_o, w_o))
  idx_o = np.arange(0,n_W*n_H*n_C*m, n_W*n_H*n_C)[:, None]
  idx_o = idx_o[:,:,np.newaxis,np.newaxis]
  offset_idx = offset_idx + idx_o
  
  # Get all actual indices & index into input array for final output

  return np.take(A,start_idx.ravel()[:,None] + offset_idx.ravel()[::stepsize])

In [ ]:
# FUNCIONES PARA LA CONVOLUCIÓN
# imgIn => (#ejemplos)x(#canales)x(ancho)x(altura)
# Filters => (#filtros)x(#canales)x(ancho)x(altura)
def conv4D(imgIn, filters, padding, stride):
  #ESTA FUNCION BOTA LA CONVOLUCIÓN PERO NO CON LA FORMA QUE DEBERÍA BOTAR
  #PERO EL CÁLCULO ES CORRECTO
  m, n_C, n_H, n_W = imgIn.shape
  n_f, f_C, f_H, f_W = filters.shape
  n = int((n_H+2*padding-f_H)/stride + 1)
  outputs = np.zeros((m,n_f,n,n))

  B = imNDcol(imgIn, [f_C, f_H, f_W])
  F = imNDcol(filters, [f_C, f_H, f_W])
  #Convolución
  CONV = B.T @ F

  return CONV